In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from datetime import datetime
import dateutil

from numpy import abs

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Input Data**:

In [ ]:
Train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv",parse_dates=["date"])

In [ ]:
Train

# **Exploratory Data Analysis**

In [ ]:
Train_plot = Train.copy()
for ind in Train_plot.index.values:
    #print(ind)
    if (Train.iloc[ind]['country'] != 'Finland' or
        Train.iloc[ind]['store'] != 'KaggleMart' or
        Train.iloc[ind]['product']  != 'Kaggle Mug'):
        Train_plot = Train.drop(ind,0)

#sns.lineplot('date','num_sold',data=Train_plot,)

 Holidays appear to affect the sales gravely.

Distribution of Data:

In [ ]:
sns.displot(Train['country'])

In [ ]:
sns.displot(Train['product'])

In [ ]:

sns.displot(data=Train, x='store', col='country', height=3, aspect=1.6)


In [ ]:
per_month = Train.copy()
per_month['month'] = Train['date'].dt.month
per_month['countrystore'] = [Train.iloc[ind]['country'] + Train.iloc[ind]['store'] for ind in Train.index.values]


sns.displot(data=per_month, x='month', y='num_sold', row='countrystore', col='product', color='red', height=3, aspect=1.6)


In [ ]:
sns.displot(data=Train,x='num_sold')

Sales distribution

## Adding Holiday and day-of-week features:

In [ ]:
Train_copy = Train.copy()
Train_copy['dayofyear'] = Train_copy['date'].dt.dayofyear
Train_copy['dayofweek'] = Train_copy['date'].dt.day_of_week

Train_copy['year'] = Train_copy['date'].dt.year
def dtchristmas(date):
    
    christmas_curr = pd.Timestamp(date.year,12,25)
    
    curr_dt = (date - christmas_curr).days
    
    if curr_dt >= -70 and curr_dt <=8:
        return curr_dt
    else:
        return 100
    
def dteaster(date):
    easter_curr = pd.Timestamp(dateutil.easter.easter(date.year))
    dt = (date - easter_curr).days
    if dt >= -1 and dt <=15:
        return dt
    else:
        return 100 
     

Train_copy['dtchristmas'] = [dtchristmas(Train_copy.iloc[ind]['date']) for ind in Train_copy.index.values]
Train_copy['dteaster'] = [dteaster(Train_copy.iloc[ind]['date']) for ind in Train_copy.index.values]
Train_copy['1jan'] = [int(Train_copy.iloc[ind]['date'] == pd.Timestamp(Train_copy.iloc[ind]['date'].year,1,1)) for ind in Train_copy.index.values]
Train_copy['wd4'] = Train_copy['date'].dt.day_of_week == 4
Train_copy['wd56'] = Train_copy['date'].dt.day_of_week >= 5
# for k in range(1, 3): # 20
#         Train_copy[f'sin{k}'] = np.sin(Train_copy['dayofyear'] / 365 * 2 * math.pi * k)
#         Train_copy[f'cos{k}'] = np.cos(Train_copy['dayofyear'] / 365 * 2 * math.pi * k)
#         Train_copy[f'mug_sin{k}'] = Train_copy[f'sin{k}'] * (Train_copy['product'] == 'Kaggle Mug')
#         Train_copy[f'mug_cos{k}'] = Train_copy[f'cos{k}'] * (Train_copy['product'] == 'Kaggle Mug')
#         Train_copy[f'sticker_sin{k}'] = Train_copy[f'sin{k}'] * (Train_copy['product'] == 'Kaggle Sticker')
#         Train_copy[f'sticker_cos{k}'] = Train_copy[f'cos{k}'] * (Train_copy['product'] == 'Kaggle Sticker')
#         Train_copy[f'sticker_sin{k}'] = Train_copy[f'sin{k}'] * (Train_copy['product'] == 'Kaggle Hat')
#         Train_copy[f'sticker_cos{k}'] = Train_copy[f'cos{k}'] * (Train_copy['product'] == 'Kaggle Hat')
Train_copy

Segregating December and first 10 days of January

In [ ]:
def remove_december(data):
    data_ = data.copy()
    for ind in data_.index.values:
        if data_.iloc[ind]['date'].month == 12:
            data_.drop(ind,axis=0,inplace=True)
    return data_
def remove_10_jan(data):
    data_ = data.copy()
    for ind in data_.index.values:
        if data_.iloc[ind]['date'].month == 1 and data_.iloc[ind]['date'].day <= 10:
            data_.drop(ind,axis=0,inplace=True)
    return data_
def remove_easter(data):
    data_ = data.copy()
    for ind in data_.index.values:
        date = data_.iloc[ind]['date']
        easter_curr = pd.Timestamp(dateutil.easter.easter(date.year))
        dt = (date - easter_curr).days
        if dt >= -1 and dt <=15:
            data_.drop(ind,axis=0,inplace=True)
    return data_

Train_dec = Train_copy.query('date.dt.month == 12')
Train_clean = pd.concat([Train_copy,Train_dec]).drop_duplicates(keep=False)
Train_10jan = Train_copy.query('date.dt.month == 1 and date.dt.day <= 10')
Train_clean = pd.concat([Train_clean,Train_10jan]).drop_duplicates(keep=False)

Train_easter = Train_copy.query('dteaster != 100')
Train_clean = pd.concat([Train_clean,Train_easter]).drop_duplicates(keep=False)




In [ ]:
Train_easter

In [ ]:
from numpy import abs
from pandas import Timestamp


sns.lineplot(x=range(1,63),y=Train_copy.query('country == "Finland" and store == "KaggleMart" and product == "Kaggle Mug" and ((date.dt.year == 2015 and ( date.dt.month ==12)) or (date.dt.year == 2016 and (date.dt.month == 1)))')['num_sold'])

In [ ]:
sns.heatmap(Train_copy.query('country == "Finland" and store == "KaggleMart" and product == "Kaggle Mug"').corr(),annot=True)



# **Fitting one part of the data**:

The plan is as follows: Each country/store/product combination is treated seperately. I get the sales data per day for each year and divide by the average sales for that year. Then a SVR is fitted to the data and the next sales average is extrapolated via linear regression.

### Extracting Data:

In [ ]:
cnt = "Finland"
stor = "KaggleMart"
prod = "Kaggle Mug"
F_KM_KM_2015 = Train_copy.query('country == @cnt and store == @stor and product == @prod and date.dt.year == 2015')
F_KM_KM_2016 = Train_copy.query('country == @cnt and store == @stor and product == @prod and date.dt.year == 2016')
F_KM_KM_2017 = Train_copy.query('country == @cnt and store == @stor and product == @prod and date.dt.year == 2017')
F_KM_KM_2018 = Train_copy.query('country == @cnt and store == @stor and product == @prod and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True) #, 'dteaster', 'dtchristmas', '1jan'
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_true_2015 = np.array(F_KM_KM_2015)
sample_x_true_2016 = np.array(F_KM_KM_2016)
sample_x_true_2017 = np.array(F_KM_KM_2017)
sample_x_true_2018 = np.array(F_KM_KM_2018)

sample_y_true_2015 = np.array(F_KM_KM_2015_y)
sample_y_true_2016 = np.array(F_KM_KM_2016_y)
sample_y_true_2017 = np.array(F_KM_KM_2017_y)
sample_y_true_2018 = np.array(F_KM_KM_2018_y)

max_true_2015 = np.max(np.array(sample_y_true_2015))
max_true_2016 = np.max(np.array(sample_y_true_2016))
max_true_2017 = np.max(np.array(sample_y_true_2017))
max_true_2018 = np.max(np.array(sample_y_true_2018))

sample_y_true_2015 = sample_y_true_2015 / max_true_2015
sample_y_true_2016 = sample_y_true_2016 / max_true_2016
sample_y_true_2017 = sample_y_true_2017 / max_true_2017
sample_y_true_2018 = sample_y_true_2018 / max_true_2018

In [ ]:

cnt = "Finland"
stor = "KaggleMart"
prod = "Kaggle Mug"
F_KM_KM_2015 = Train_clean.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2015')
F_KM_KM_2016 = Train_clean.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2016')
F_KM_KM_2017 = Train_clean.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2017')
F_KM_KM_2018 = Train_clean.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_2015 = np.array(F_KM_KM_2015)
sample_x_2016 = np.array(F_KM_KM_2016)
sample_x_2017 = np.array(F_KM_KM_2017)
sample_x_2018 = np.array(F_KM_KM_2018)

sample_y_2015 = np.array(F_KM_KM_2015_y)
sample_y_2016 = np.array(F_KM_KM_2016_y)
sample_y_2017 = np.array(F_KM_KM_2017_y)
sample_y_2018 = np.array(F_KM_KM_2018_y)

sample_y_2015 = sample_y_2015 / max_true_2015
sample_y_2016 = sample_y_2016 / max_true_2016
sample_y_2017 = sample_y_2017 / max_true_2017
sample_y_2018 = sample_y_2018 / max_true_2018

In [ ]:

cnt = "Finland"
stor = "KaggleMart"
prod = "Kaggle Mug"
F_KM_KM_2015 = Train_dec.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2015')
F_KM_KM_2016 = Train_dec.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2016')
F_KM_KM_2017 = Train_dec.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2017')
F_KM_KM_2018 = Train_dec.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_dec_2015 = np.array(F_KM_KM_2015)
sample_x_dec_2016 = np.array(F_KM_KM_2016)
sample_x_dec_2017 = np.array(F_KM_KM_2017)
sample_x_dec_2018 = np.array(F_KM_KM_2018)

sample_y_dec_2015 = np.array(F_KM_KM_2015_y)
sample_y_dec_2016 = np.array(F_KM_KM_2016_y)
sample_y_dec_2017 = np.array(F_KM_KM_2017_y)
sample_y_dec_2018 = np.array(F_KM_KM_2018_y)


In [ ]:
cnt = "Finland"
stor = "KaggleMart"
prod = "Kaggle Mug"
F_KM_KM_2015 = Train_easter.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2015')
F_KM_KM_2016 = Train_easter.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2016')
F_KM_KM_2017 = Train_easter.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2017')
F_KM_KM_2018 = Train_easter.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_easter_2015 = np.array(F_KM_KM_2015)
sample_x_easter_2016 = np.array(F_KM_KM_2016)
sample_x_easter_2017 = np.array(F_KM_KM_2017)
sample_x_easter_2018 = np.array(F_KM_KM_2018)

sample_y_easter_2015 = np.array(F_KM_KM_2015_y)
sample_y_easter_2016 = np.array(F_KM_KM_2016_y)
sample_y_easter_2017 = np.array(F_KM_KM_2017_y)
sample_y_easter_2018 = np.array(F_KM_KM_2018_y)

In [ ]:
cnt = "Finland"
stor = "KaggleMart"
prod = "Kaggle Mug"
F_KM_KM_2015 = Train_10jan.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2015')
F_KM_KM_2016 = Train_10jan.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2016')
F_KM_KM_2017 = Train_10jan.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2017')
F_KM_KM_2018 = Train_10jan.query('country == @cnt and store == "KaggleMart" and product == "Kaggle Mug" and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_10jan_2015 = np.array(F_KM_KM_2015)
sample_x_10jan_2016 = np.array(F_KM_KM_2016)
sample_x_10jan_2017 = np.array(F_KM_KM_2017)
sample_x_10jan_2018 = np.array(F_KM_KM_2018)


In [ ]:
sample_x = np.concatenate((sample_x_2017,sample_x_2016,sample_x_2015),axis = 0)
sample_y = np.concatenate((sample_y_2017,sample_y_2016,sample_y_2015),axis = 0)

sample_x_true = np.concatenate((sample_x_true_2017,sample_x_true_2016,sample_x_true_2015),axis = 0)
sample_y_true = np.concatenate((sample_y_true_2017,sample_y_true_2016,sample_y_true_2015),axis = 0)

sample_x_dec = np.concatenate((sample_x_dec_2017,sample_x_dec_2016,sample_x_dec_2015),axis = 0)
sample_y_dec = np.concatenate((sample_y_dec_2017,sample_y_dec_2016,sample_y_dec_2015),axis = 0)

sample_x_easter = np.concatenate((sample_x_easter_2017,sample_x_easter_2016,sample_x_easter_2015),axis = 0)
sample_x_10jan = np.concatenate((sample_x_10jan_2017,sample_x_10jan_2016,sample_x_10jan_2015),axis = 0)

### Fitting a SVR to the data:

In [ ]:
from sklearn import svm
from sklearn.linear_model import HuberRegressor, LinearRegression, Ridge
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

def chosen_model():
    #return Ridge()
    return GradientBoostingRegressor(loss="huber",max_depth=4,n_estimators=70,alpha=0.95,random_state=1)
    #return svm.SVR(kernel="rbf", C=10000, gamma=0.25, epsilon=1e-10, tol=1e-10)
    #return AdaBoostRegressor(n_estimators = 1000, loss='square', random_state = 1)
sample_regr = chosen_model()
sample_regr.fit(sample_x, sample_y)


In [ ]:
deltay_2018 = sample_y_true_2018-sample_regr.predict(sample_x_true_2018)
deltay = sample_y_true-sample_regr.predict(sample_x_true)
g = sns.lineplot(range(1,1097),deltay)
g.axhline(0,color='red')

We can see that predictibly the largest residual is situated at the hollidays.

In [ ]:
len(deltay)

### Correcting for December

In [ ]:
deltay_dec = np.zeros(len(sample_x_dec))
for i in range(len(sample_x_dec)):
    for j in range(len(sample_x_true)):
        if np.all(sample_x_true[j] == sample_x_dec[i]):
            deltay_dec[i] = deltay[j]
            break



In [ ]:
sns.lineplot(range(1,94),deltay_dec)

In [ ]:
deltay_dec_2017 = np.zeros(31)
j=0
for i in range(len(sample_x_dec)):
    if sample_x_dec[i][2] == 2017:
        deltay_dec_2017[j] = deltay_dec[i]
        j += 1



In [ ]:
sns.lineplot(range(1,32),deltay_dec_2017)

In [ ]:
dec_regr = chosen_model()
dec_regr.fit(sample_x_dec, deltay_dec)

In [ ]:
def predict_dec(sample,sample_dec):  
    y = np.zeros(len(sample))
    y = sample_regr.predict(sample)
    dec_residual = dec_regr.predict(sample_dec)
    
    for i in range(len(sample)):
        if sample[i][0] >= 335:
            for j in range(len(sample_dec)):
                if np.all(sample[i] == sample_dec[j]):
                    y[i] +=  dec_residual[j]
                    break
    return y

In [ ]:
dec_predict_2018 = predict_dec(sample_x_true_2018,sample_x_dec_2018)

len(dec_predict_2018)
sns.lineplot(range(1,366),dec_predict_2018)
sns.lineplot(range(1,366),sample_y_true_2018,style=2)

In [ ]:
dec_predict = predict_dec(sample_x_true,sample_x_dec)
deltay = sample_y_true - dec_predict #Updating residuals
sns.lineplot(range(1,1097),deltay)

### Correcting for Easter

In [ ]:
deltay_easter = np.zeros(len(sample_x_easter))
for i in range(len(sample_x_easter)):
    for j in range(len(sample_x_true)):
        if np.all(sample_x_true[j] == sample_x_easter[i]):
            deltay_easter[i] = deltay[j]
            break



In [ ]:
deltay_easter_2017 = np.array([])
j=0
for i in range(len(sample_x_easter)):
    if sample_x_easter[i][2] == 2017:
        deltay_easter_2017 = np.append(deltay_easter_2017,deltay_easter[i])
        j += 1

len(deltay_easter_2017)

In [ ]:
sns.lineplot(range(17),deltay_easter_2017)

In [ ]:
easter_regr = chosen_model()
easter_regr.fit(sample_x_easter, deltay_easter)

In [ ]:
sample_x_easter_2018

In [ ]:
def predict_easter(sample,sample_easter,y):  
    y = y.copy()
    #y = predict_dec(sample,sample_dec)
    easter_residual = easter_regr.predict(sample_easter)
    
    for i in range(len(sample)):
        if sample[i][0] >= 86 and sample[i][0] <= 121:
            for j in range(len(sample_easter)):
                if np.all(sample[i] == sample_easter[j]):
                    y[i] +=  easter_residual[j]
                    break
    return y

In [ ]:
easter_predict_2018 = predict_easter(sample_x_true_2018,sample_x_easter_2018,dec_predict_2018)

len(easter_predict_2018)
sns.lineplot(range(1,366),easter_predict_2018)
sns.lineplot(range(1,366),sample_y_true_2018,style=2)

In [ ]:
easter_predict = predict_easter(sample_x_true,sample_x_easter, dec_predict)
deltay = sample_y_true - easter_predict #Updating residuals

In [ ]:
sns.lineplot(range(1,1097), easter_predict)

In [ ]:
deltay_10jan = np.zeros(len(sample_x_10jan))
for i in range(len(sample_x_10jan)):
    for j in range(len(sample_x_true)):
        if np.all(sample_x_true[j] == sample_x_10jan[i]):
            deltay_10jan[i] = deltay[j]
            break

len(deltay_10jan)

In [ ]:
sns.lineplot(range(30),deltay_10jan)

In [ ]:
deltay_10jan_2017 = np.array([])
j=0
for i in range(len(sample_x_10jan)):
    if sample_x_10jan[i][2] == 2017:
        deltay_10jan_2017 = np.append(deltay_10jan_2017,deltay_10jan[i])
        j += 1

len(deltay_10jan_2017)

In [ ]:
sns.lineplot(range(10),deltay_10jan_2017)

In [ ]:
jan10_regr = chosen_model()
jan10_regr.fit(sample_x_10jan, deltay_10jan)

In [ ]:
def predict_10jan(sample,sample_jan10,y):  
    y = y.copy()
    jan10_residual = jan10_regr.predict(sample_jan10)
    
    for i in range(len(sample)):
        if sample[i][0] <= 10:
            for j in range(len(sample_jan10)):
                if np.all(sample[i] == sample_jan10[j]):
                    y[i] +=  jan10_residual[j]
                    break
    return y

In [ ]:
jan10_predict_2018 = predict_10jan(sample_x_10jan_2018,sample_x_10jan_2018,easter_predict_2018)
print(np.all(jan10_predict_2018 == easter_predict_2018))
len(jan10_predict_2018)
sns.lineplot(range(1,366),jan10_predict_2018)
#sns.lineplot(range(1,366),easter_predict_2018)
#sns.lineplot(range(1,366),sample_y_true_2018,style=2)

In [ ]:
from sklearn.metrics import mean_squared_error

def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.round(np.mean(diff),5)

#print(mean_squared_error(sample_y_2018,sample_mean))
print(SMAPE(sample_y_true_2018,(sample_regr.predict(sample_x_true_2018))))
print(SMAPE(sample_y_true_2018,dec_predict_2018))
print(SMAPE(sample_y_true_2018,easter_predict_2018))
print(SMAPE(sample_y_true_2018,jan10_predict_2018))
print(mean_squared_error(sample_y_2018,sample_regr.predict(sample_x_2018)))

The error is decent.

Fitting the Linear regressor for the year-by-year average

In [ ]:
from sklearn.linear_model import LinearRegression

max_model = LinearRegression()
max_model.fit(np.array(range(2015,2018)).reshape(-1,1),[max_true_2015,max_true_2016,max_true_2017])

(max_model.predict(np.array(2018).reshape(1,-1))[0])



Seems Good! Now applying to to all country/store/product combinations (3x2x3 = 18 different combinations)

In [ ]:
country = 'Norway'
store = 'KaggleMart'
product = 'Kaggle Sticker'


F_KM_KM_2015 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
F_KM_KM_2016 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
F_KM_KM_2017 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
F_KM_KM_2018 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True) #, 'dteaster', 'dtchristmas', '1jan'
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_true_2015 = np.array(F_KM_KM_2015)
sample_x_true_2016 = np.array(F_KM_KM_2016)
sample_x_true_2017 = np.array(F_KM_KM_2017)
sample_x_true_2018 = np.array(F_KM_KM_2018)

sample_y_true_2015 = np.array(F_KM_KM_2015_y)
sample_y_true_2016 = np.array(F_KM_KM_2016_y)
sample_y_true_2017 = np.array(F_KM_KM_2017_y)
sample_y_true_2018 = np.array(F_KM_KM_2018_y)

max_true_2015 = np.max(np.array(sample_y_true_2015))
max_true_2016 = np.max(np.array(sample_y_true_2016))
max_true_2017 = np.max(np.array(sample_y_true_2017))
max_true_2018 = np.max(np.array(sample_y_true_2018))

sample_y_true_2015 = sample_y_true_2015 / max_true_2015
sample_y_true_2016 = sample_y_true_2016 / max_true_2016
sample_y_true_2017 = sample_y_true_2017 / max_true_2017
sample_y_true_2018 = sample_y_true_2018 / max_true_2018

F_KM_KM_2015 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
F_KM_KM_2016 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
F_KM_KM_2017 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
F_KM_KM_2018 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_2015 = np.array(F_KM_KM_2015)
sample_x_2016 = np.array(F_KM_KM_2016)
sample_x_2017 = np.array(F_KM_KM_2017)
sample_x_2018 = np.array(F_KM_KM_2018)

sample_y_2015 = np.array(F_KM_KM_2015_y)
sample_y_2016 = np.array(F_KM_KM_2016_y)
sample_y_2017 = np.array(F_KM_KM_2017_y)
sample_y_2018 = np.array(F_KM_KM_2018_y)

sample_y_2015 = sample_y_2015 / max_true_2015
sample_y_2016 = sample_y_2016 / max_true_2016
sample_y_2017 = sample_y_2017 / max_true_2017
sample_y_2018 = sample_y_2018 / max_true_2018

F_KM_KM_2015 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
F_KM_KM_2016 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
F_KM_KM_2017 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
F_KM_KM_2018 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_dec_2015 = np.array(F_KM_KM_2015)
sample_x_dec_2016 = np.array(F_KM_KM_2016)
sample_x_dec_2017 = np.array(F_KM_KM_2017)
sample_x_dec_2018 = np.array(F_KM_KM_2018)

F_KM_KM_2015 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
F_KM_KM_2016 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
F_KM_KM_2017 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
F_KM_KM_2018 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

sample_x_10jan_2015 = np.array(F_KM_KM_2015)
sample_x_10jan_2016 = np.array(F_KM_KM_2016)
sample_x_10jan_2017 = np.array(F_KM_KM_2017)
sample_x_10jan_2018 = np.array(F_KM_KM_2018)

sample_x = np.concatenate((sample_x_2017,sample_x_2016,sample_x_2015),axis = 0)
sample_y = np.concatenate((sample_y_2017,sample_y_2016,sample_y_2015),axis = 0)

sample_x_true = np.concatenate((sample_x_true_2017,sample_x_true_2016,sample_x_true_2015),axis = 0)
sample_y_true = np.concatenate((sample_y_true_2017,sample_y_true_2016,sample_y_true_2015),axis = 0)

sample_x_dec = np.concatenate((sample_x_dec_2017,sample_x_dec_2016,sample_x_dec_2015),axis = 0)
sample_y_dec = np.concatenate((sample_y_dec_2017,sample_y_dec_2016,sample_y_dec_2015),axis = 0)

sample_x_easter = np.concatenate((sample_x_easter_2017,sample_x_easter_2016,sample_x_easter_2015),axis = 0)
sample_x_10jan = np.concatenate((sample_x_10jan_2017,sample_x_10jan_2016,sample_x_10jan_2015),axis = 0)


sample_regr = chosen_model()
sample_regr.fit(sample_x, sample_y)

deltay = sample_y_true-sample_regr.predict(sample_x_true)

deltay_dec = np.zeros(len(sample_x_dec))
for i in range(len(sample_x_dec)):
    for j in range(len(sample_x_true)):
        if np.all(sample_x_true[j] == sample_x_dec[i]):
            deltay_dec[i] = deltay[j]
            break

dec_regr = chosen_model()
dec_regr.fit(sample_x_dec, deltay_dec)



dec_predict = predict_dec(sample_x_true,sample_x_dec)
deltay = sample_y_true - dec_predict #Updating residuals

deltay_easter = np.zeros(len(sample_x_easter))
for i in range(len(sample_x_easter)):
    for j in range(len(sample_x_true)):
        if np.all(sample_x_true[j] == sample_x_easter[i]):
            deltay_easter[i] = deltay[j]
            break

easter_regr = chosen_model()
easter_regr.fit(sample_x_easter, deltay_easter)


easter_predict = predict_easter(sample_x_true,sample_x_easter, dec_predict)
deltay = sample_y_true - easter_predict #Updating residuals

deltay_10jan = np.zeros(len(sample_x_10jan))
for i in range(len(sample_x_10jan)):
    for j in range(len(sample_x_true)):
        if np.all(sample_x_true[j] == sample_x_10jan[i]):
            deltay_10jan[i] = deltay[j]
            break


jan10_regr = chosen_model()
jan10_regr.fit(sample_x_10jan, deltay_10jan)


max_model = LinearRegression()
max_model.fit(np.array(range(2015,2019)).reshape(-1,1),[max_true_2015,max_true_2016,max_true_2017,max_true_2018])


In [ ]:
# country = 'Finland'
# store = 'KaggleRama'
# product = 'Kaggle Mug'


# sample_regr, dec_regr, easter_regr, jan10_regr, max_model = model[country][store][product]#learn_product(country,store,product)

# F_KM_KM_2018 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2018')
# F_KM_KM_2018_y = F_KM_KM_2018['num_sold']
# F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
# sample_x_true_2018 = np.array(F_KM_KM_2018)
# sample_y_true_2018 = np.array(F_KM_KM_2018_y)
# max_true_2018 = np.max(np.array(sample_y_true_2018))
# sample_y_true_2018 = sample_y_true_2018 / max_true_2018

# row_arr = sample_x_true_2018
# #y = np.zeros(len(row_arr))
# #for i = range(len(row_arr))

# y = sample_regr.predict(row_arr)
# for i in range(len(row_arr)):
#     if row_arr[i][0] >= 335:
#         y[i] = y[i] + dec_regr.predict([row_arr[i]])
#     elif row_arr[i][0] <= 10:
#         y[i] += jan10_regr.predict([row_arr[i]])
#     elif row_arr[0][4] != 100:
#         y[i] += easter_regr.predict([row_arr[i]])
# year_max = max_model.predict([[2018]])[0]

# #dec_predict_2018 = predict_dec(sample_x_true_2018,sample_x_dec_2018)
# #easter_predict_2018 = predict_easter(sample_x_true_2018,sample_x_easter_2018,dec_predict_2018)
# #sns.lineplot(x=range(1,366),y=easter_predict_2018)
# #y = predict_10jan(sample_x_true_2018,sample_x_10jan_2018,easter_predict_2018)

# print(year_max)

# #
# sns.lineplot(x=range(1,366),y=y)
# #sns.lineplot(x=range(1,366),y=dec_regr.predict(row_arr))
# #sns.lineplot(x=range(1,366),y=dec_regr.predict(row_arr) + y)
# sns.lineplot(x=range(1,366),y=sample_y_true_2018)
# print(SMAPE(sample_y_true_2018*max_true_2018,y*year_max))

In [ ]:
def learn_product(country,store,product):

    F_KM_KM_2015 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
    F_KM_KM_2016 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
    F_KM_KM_2017 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
    F_KM_KM_2018 = Train_copy.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

    F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
    F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
    F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
    F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

    F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True) #, 'dteaster', 'dtchristmas', '1jan'
    F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

    sample_x_true_2015 = np.array(F_KM_KM_2015)
    sample_x_true_2016 = np.array(F_KM_KM_2016)
    sample_x_true_2017 = np.array(F_KM_KM_2017)
    sample_x_true_2018 = np.array(F_KM_KM_2018)

    sample_y_true_2015 = np.array(F_KM_KM_2015_y)
    sample_y_true_2016 = np.array(F_KM_KM_2016_y)
    sample_y_true_2017 = np.array(F_KM_KM_2017_y)
    sample_y_true_2018 = np.array(F_KM_KM_2018_y)

    max_true_2015 = np.max(np.array(sample_y_true_2015))
    max_true_2016 = np.max(np.array(sample_y_true_2016))
    max_true_2017 = np.max(np.array(sample_y_true_2017))
    max_true_2018 = np.max(np.array(sample_y_true_2018))

    sample_y_true_2015 = sample_y_true_2015 / max_true_2015
    sample_y_true_2016 = sample_y_true_2016 / max_true_2016
    sample_y_true_2017 = sample_y_true_2017 / max_true_2017
    sample_y_true_2018 = sample_y_true_2018 / max_true_2018

    F_KM_KM_2015 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
    F_KM_KM_2016 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
    F_KM_KM_2017 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
    F_KM_KM_2018 = Train_clean.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

    F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
    F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
    F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
    F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

    F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

    sample_x_2015 = np.array(F_KM_KM_2015)
    sample_x_2016 = np.array(F_KM_KM_2016)
    sample_x_2017 = np.array(F_KM_KM_2017)
    sample_x_2018 = np.array(F_KM_KM_2018)

    sample_y_2015 = np.array(F_KM_KM_2015_y)
    sample_y_2016 = np.array(F_KM_KM_2016_y)
    sample_y_2017 = np.array(F_KM_KM_2017_y)
    sample_y_2018 = np.array(F_KM_KM_2018_y)

    sample_y_2015 = sample_y_2015 / max_true_2015
    sample_y_2016 = sample_y_2016 / max_true_2016
    sample_y_2017 = sample_y_2017 / max_true_2017
    sample_y_2018 = sample_y_2018 / max_true_2018

    F_KM_KM_2015 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
    F_KM_KM_2016 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
    F_KM_KM_2017 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
    F_KM_KM_2018 = Train_dec.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

    F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
    F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
    F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
    F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

    F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

    sample_x_dec_2015 = np.array(F_KM_KM_2015)
    sample_x_dec_2016 = np.array(F_KM_KM_2016)
    sample_x_dec_2017 = np.array(F_KM_KM_2017)
    sample_x_dec_2018 = np.array(F_KM_KM_2018)

    F_KM_KM_2015 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2015')
    F_KM_KM_2016 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2016')
    F_KM_KM_2017 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2017')
    F_KM_KM_2018 = Train_10jan.query('country == @country and store == @store and product == @product and date.dt.year == 2018')

    F_KM_KM_2015_y = F_KM_KM_2015['num_sold']
    F_KM_KM_2016_y = F_KM_KM_2016['num_sold']
    F_KM_KM_2017_y = F_KM_KM_2017['num_sold']
    F_KM_KM_2018_y = F_KM_KM_2018['num_sold']

    F_KM_KM_2015.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2016.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2017.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)
    F_KM_KM_2018.drop(['date', 'country', 'store', 'product', 'row_id', 'num_sold'],axis=1,inplace=True)

    sample_x_10jan_2015 = np.array(F_KM_KM_2015)
    sample_x_10jan_2016 = np.array(F_KM_KM_2016)
    sample_x_10jan_2017 = np.array(F_KM_KM_2017)
    sample_x_10jan_2018 = np.array(F_KM_KM_2018)

    sample_x = np.concatenate((sample_x_2017,sample_x_2016,sample_x_2015),axis = 0)
    sample_y = np.concatenate((sample_y_2017,sample_y_2016,sample_y_2015),axis = 0)

    sample_x_true = np.concatenate((sample_x_true_2017,sample_x_true_2016,sample_x_true_2015),axis = 0)
    sample_y_true = np.concatenate((sample_y_true_2017,sample_y_true_2016,sample_y_true_2015),axis = 0)

    sample_x_dec = np.concatenate((sample_x_dec_2017,sample_x_dec_2016,sample_x_dec_2015),axis = 0)
    sample_y_dec = np.concatenate((sample_y_dec_2017,sample_y_dec_2016,sample_y_dec_2015),axis = 0)

    sample_x_easter = np.concatenate((sample_x_easter_2017,sample_x_easter_2016,sample_x_easter_2015),axis = 0)
    sample_x_10jan = np.concatenate((sample_x_10jan_2017,sample_x_10jan_2016,sample_x_10jan_2015),axis = 0)


    sample_regr = chosen_model()
    sample_regr.fit(sample_x, sample_y)

    deltay = sample_y_true-sample_regr.predict(sample_x_true)

    deltay_dec = np.zeros(len(sample_x_dec))
    for i in range(len(sample_x_dec)):
        for j in range(len(sample_x_true)):
            if np.all(sample_x_true[j] == sample_x_dec[i]):
                deltay_dec[i] = deltay[j]
                break

    dec_regr = chosen_model()
    dec_regr.fit(sample_x_dec, deltay_dec)



    dec_predict = predict_dec(sample_x_true,sample_x_dec)
    deltay = sample_y_true - dec_predict #Updating residuals

    deltay_easter = np.zeros(len(sample_x_easter))
    for i in range(len(sample_x_easter)):
        for j in range(len(sample_x_true)):
            if np.all(sample_x_true[j] == sample_x_easter[i]):
                deltay_easter[i] = deltay[j]
                break

    easter_regr = chosen_model()
    easter_regr.fit(sample_x_easter, deltay_easter)


    easter_predict = predict_easter(sample_x_true,sample_x_easter, dec_predict)
    deltay = sample_y_true - easter_predict #Updating residuals

    deltay_10jan = np.zeros(len(sample_x_10jan))
    for i in range(len(sample_x_10jan)):
        for j in range(len(sample_x_true)):
            if np.all(sample_x_true[j] == sample_x_10jan[i]):
                deltay_10jan[i] = deltay[j]
                break


    jan10_regr = chosen_model()
    jan10_regr.fit(sample_x_10jan, deltay_10jan)


    max_model = LinearRegression()
    max_model.fit(np.array(range(2015,2019)).reshape(-1,1),[max_true_2015,max_true_2016,max_true_2017,max_true_2018])

    return sample_regr, dec_regr, easter_regr, jan10_regr, max_model

def fit_model():
    country_dict = {}
    for country in ['Finland','Sweden','Norway']:
        curr_country_dict = {}
        for store in ['KaggleMart','KaggleRama']:
            curr_store_dict = {}
            for product in ['Kaggle Mug','Kaggle Sticker', 'Kaggle Hat']:
                curr_store_dict[product] = learn_product(country,store,product)
            curr_country_dict[store] = curr_store_dict
        country_dict[country] = curr_country_dict
    return country_dict

In [ ]:
model = fit_model()

## Prediction

In [ ]:
Test  = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv",parse_dates=["date"])


Test_copy = Test.copy()
Test_copy['dayofyear'] = Test_copy['date'].dt.dayofyear
Test_copy['dayofweek'] = Test_copy['date'].dt.day_of_week

Test_copy['year'] = Test_copy['date'].dt.year

Test_copy['dtchristmas'] = [dtchristmas(Test_copy.iloc[ind]['date']) for ind in Test_copy.index.values]
Test_copy['dteaster'] = [dteaster(Test_copy.iloc[ind]['date']) for ind in Test_copy.index.values]
Test_copy['1jan'] = [int(Test_copy.iloc[ind]['date'] == pd.Timestamp(Test_copy.iloc[ind]['date'].year,1,1)) for ind in Test_copy.index.values]
Test_copy['wd4'] = Test_copy['date'].dt.day_of_week == 4
Test_copy['wd56'] = Test_copy['date'].dt.day_of_week >= 5
# for k in range(1, 3): # 20
#         Test_copy[f'sin{k}'] = np.sin(Test_copy['dayofyear'] / 365 * 2 * math.pi * k)
#         Test_copy[f'cos{k}'] = np.cos(Test_copy['dayofyear'] / 365 * 2 * math.pi * k)
#         Test_copy[f'mug_sin{k}'] = Test_copy[f'sin{k}'] * (Test_copy['product'] == 'Kaggle Mug')
#         Test_copy[f'mug_cos{k}'] = Test_copy[f'cos{k}'] * (Test_copy['product'] == 'Kaggle Mug')
#         Test_copy[f'sticker_sin{k}'] = Test_copy[f'sin{k}'] * (Test_copy['product'] == 'Kaggle Sticker')
#         Test_copy[f'sticker_cos{k}'] = Test_copy[f'cos{k}'] * (Test_copy['product'] == 'Kaggle Sticker')
#         Test_copy[f'sticker_sin{k}'] = Test_copy[f'sin{k}'] * (Test_copy['product'] == 'Kaggle Hat')
#         Test_copy[f'sticker_cos{k}'] = Test_copy[f'cos{k}'] * (Test_copy['product'] == 'Kaggle Hat')
        

In [ ]:
def predict_data(model,row):
    country = row['country']
    store = row['store']
    product = row['product']
    #return country,store,product
    sample_regr, dec_regr, easter_regr, jan10_regr, max_model = model[country][store][product] 
    
    row_arr = row.copy()
    row_arr.drop(['date', 'country', 'store', 'product', 'row_id'],inplace=True)
    row_arr = np.array(row_arr)
    row_arr = [row_arr]
    
    #product_model = sample_regr, dec_regr, easter_regr, jan10_regr, max_model
    #return row_arr
    y = sample_regr.predict(row_arr)
    if row_arr[0][0] >= 335:
        y += dec_regr.predict(row_arr)
    elif row_arr[0][0] <= 10:
        y += jan10_regr.predict(row_arr)
    elif row_arr[0][4] != 100:
        y += easter_regr.predict(row_arr)
    year_max = max_model.predict([[2018]])[0]
    
    return y[0] * year_max
    

In [ ]:
#SMAPE(Train_copy['num_sold'],[predict_data(model,Train_copy.iloc[ind]) for ind in Train_copy.index.values])

## Submission

In [ ]:
Result_DF = pd.DataFrame()
Result_DF['row_id'] = Test['row_id']
Result_DF['num_sold'] = [predict_data(model,Test_copy.iloc[ind]) for ind in Test_copy.index.values]

In [ ]:
Result_DF['num_sold'] = np.round(Result_DF['num_sold']).astype(int)

In [ ]:
Result_DF

In [ ]:
Result_DF.to_csv('submission.csv',index=False)